<div class="alert alert-info alert-warning" style="background-color: white; color: black; text-align: center;">
    <h1><span style="color: red;">Ozan MÖHÜRCÜ</span></h1>
    <h1><span style="color: red;">Data Analyst | Data Scientist</span></h1>

 <div style="text-align: center; font-family: Arial, sans-serif; margin-top: 20px;">
        <a href="https://www.linkedin.com/in/ozanmhrc/" style="text-decoration: none; color: #fff; margin-right: 10px;">
            <span style="background-color: #0077B5; padding: 8px 20px; border-radius: 5px; font-size: 14px; display: inline-block; width: 120px; text-align: center;">LinkedIn</span>
        </a>
        <a href="https://github.com/Ozan-Mohurcu" style="text-decoration: none; color: #fff; margin-right: 10px;">
            <span style="background-color: #333; padding: 8px 20px; border-radius: 5px; font-size: 14px; display: inline-block; width: 120px; text-align: center;">GitHub</span>
        </a>
    </div>
</div>

<div style="background-color: white; color: black; padding: 20px; border-radius: 8px;">
  <h2 style="color: red;">Libraries Import</h2>
</div>

In [1]:
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
from itertools import combinations
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, QuantileTransformer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_classif
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
import optuna
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

warnings.filterwarnings("ignore")

<div style="background-color: white; color: black; padding: 20px; border-radius: 8px;">
  <h2 style="color: red;">Data Loading</h2>
</div>

In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e6/test.csv")
original = pd.read_csv("/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv")

In [3]:
def rename_temperature_column(df):
    df = df.rename(columns={'Temparature': 'Temperature'})
    return df

train = rename_temperature_column(train)
test = rename_temperature_column(test)
original = rename_temperature_column(original)

# Orijinal veri ile birleştirme
print(f"Train boyutu: {train.shape}")
print(f"Original boyutu: {original.shape}")
train = pd.concat([train, original], axis=0, ignore_index=True)
print(f"Birleştirilmiş train boyutu: {train.shape}")

Train boyutu: (750000, 10)
Original boyutu: (100000, 9)
Birleştirilmiş train boyutu: (850000, 10)


In [4]:
print(train.columns)

Index(['id', 'Temperature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type',
       'Nitrogen', 'Potassium', 'Phosphorous', 'Fertilizer Name'],
      dtype='object')


<div style="background-color: white; color: black; padding: 20px; border-radius: 8px;">
  <h2 style="color: red;">Feature Engineering</h2>
  <p>
      
- Feature Engineering is the process of creating meaningful input features from raw data to improve the performance of machine learning models.  
- It involves transforming, selecting, or generating new features.  
- Good feature engineering can significantly enhance model accuracy and efficiency.  
- Even simple models can perform well with well-crafted features.
  </p>
</div>

In [5]:
def advanced_feature_engineering(df, is_train=True):
    df = df.copy()
    
    df['temp_category'] = pd.cut(df['Temperature'], bins=5, labels=['very_cold', 'cold', 'moderate', 'warm', 'hot'])
    
    
    df['humidity_category'] = pd.cut(df['Humidity'], bins=4, labels=['low', 'medium', 'high', 'very_high'])
    

    
    df['temp_humidity_ratio'] = df['Temperature'] / (df['Humidity'] + 1)
    df['npk_sum'] = df['Nitrogen'] + df['Phosphorous'] + df['Potassium']
    df['npk_ratio_n'] = df['Nitrogen'] / (df['npk_sum'] + 1)
    df['npk_ratio_p'] = df['Phosphorous'] / (df['npk_sum'] + 1)
    df['npk_ratio_k'] = df['Potassium'] / (df['npk_sum'] + 1)
    
    
    df['climate_score'] = df['Temperature'] * df['Humidity'] / 100
    df['soil_fertility'] = (df['Nitrogen'] * df['Phosphorous'] * df['Potassium']) ** (1/3)
    
    
    df['temp_squared'] = df['Temperature'] ** 2
    df['nitrogen_log'] = np.log1p(df['Nitrogen'])
    df['phosphorous_log'] = np.log1p(df['Phosphorous'])
    df['potassium_log'] = np.log1p(df['Potassium'])
    
   
    df['dap_score'] = (df['Nitrogen'] + df['Phosphorous']) * (df['Temperature'] > 20).astype(int)
    df['potash_score'] = df['Potassium'] * (df['Soil Type'] == 'Sandy').astype(int)
    
   
    if is_train:
        
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        numeric_cols = [col for col in numeric_cols if col not in ['id']]
        
        
        global kmeans_model
        kmeans_model = KMeans(n_clusters=8, random_state=42)
        df['cluster'] = kmeans_model.fit_predict(df[numeric_cols].fillna(0))
    else:
        
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        numeric_cols = [col for col in numeric_cols if col not in ['id']]
        df['cluster'] = kmeans_model.predict(df[numeric_cols].fillna(0))
    
    return df


train_fe = advanced_feature_engineering(train, is_train=True)
test_fe = advanced_feature_engineering(test, is_train=False)

<div style="background-color: white; color: black; padding: 20px; border-radius: 8px;">
  <h2 style="color: red;">Encoding Techniques</h2>
  <p>
    - Encoding is the process of converting categorical variables into numerical format.<br>
    - It allows machine learning algorithms to interpret non-numeric data effectively.<br>
    - Common techniques include Label Encoding, One-Hot Encoding, and Target Encoding.<br>
    - Choosing the right encoding method depends on the model and data distribution.
  </p>
</div>

In [6]:
def encode_categorical_features(train_df, test_df, target_col):
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()
    
    
    cat_cols = [col for col in train_encoded.select_dtypes(include=['object', 'category']).columns 
                if col not in [target_col, 'id']]
    
    label_encoders = {}
    
    for col in cat_cols:
        le = LabelEncoder()
        
        
        combined_values = pd.concat([train_encoded[col], test_encoded[col]]).astype(str)
        le.fit(combined_values)
        
        train_encoded[col] = le.transform(train_encoded[col].astype(str))
        test_encoded[col] = le.transform(test_encoded[col].astype(str))
        
        label_encoders[col] = le
    
    return train_encoded, test_encoded, label_encoders

train_encoded, test_encoded, label_encoders = encode_categorical_features(train_fe, test_fe, "Fertilizer Name")


target_encoder = LabelEncoder()
train_encoded["Fertilizer Name"] = target_encoder.fit_transform(train_encoded["Fertilizer Name"])

In [7]:
feature_cols = [col for col in train_encoded.columns if col not in ['id', 'Fertilizer Name']]
X = train_encoded[feature_cols]
y = train_encoded["Fertilizer Name"]
X_test = test_encoded[feature_cols]

print(f"Number of features: {len(feature_cols)}")
print(f"Number of target classes: {y.nunique()}")

Number of features: 24
Number of target classes: 7


<div style="background-color: white; color: black; padding: 20px; border-radius: 8px;">
  <h2 style="color: red;">MAP@3 Evaluation Metric</h2>
  <p>
    - MAP@3 (Mean Average Precision at 3) is a ranking-based evaluation metric used in multi-class problems.<br>
    - It measures how well the top 3 predictions match the actual label.<br>
    - The score increases when the correct label is ranked higher.<br>
    - It is especially useful when only a few top predictions matter (e.g., recommendation systems).
  </p>
</div>

In [8]:
# MAP@3 metriği
def mapk(actual, predicted, k=3):
    def apk(a, p, k):
        p = p[:k]
        score = 0.0
        hits = 0
        seen = set()
        for i, pred in enumerate(p):
            if pred in a and pred not in seen:
                hits += 1
                score += hits / (i + 1.0)
                seen.add(pred)
        return score / min(len(a), k)
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

In [9]:
xgb_params = {
    'max_depth': 8,
    'colsample_bytree': 0.8,
    'subsample': 0.9,
    'n_estimators': 2000,
    'learning_rate': 0.05,
    'gamma': 0.1,
    'reg_alpha': 1.0,
    'reg_lambda': 1.5,
    'objective': 'multi:softproba',
    'random_state': 42,
    'n_jobs': -1,
    'tree_method': 'gpu_hist',       # GPU kullanımı için
    'predictor': 'gpu_predictor'     # GPU'da tahmin
}

In [10]:
lgb_params = {
    'objective': 'multiclass',
    'num_class': y.nunique(),
    'boosting_type': 'gbdt',
    'num_leaves': 64,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 5,
    'min_child_samples': 20,
    'reg_alpha': 0.5,
    'reg_lambda': 0.5,
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1,
    'device': 'gpu',                  # GPU kullanımını etkinleştirir
}

In [11]:
cat_params = {
    'iterations': 2000,
    'learning_rate': 0.05,
    'depth': 8,
    'l2_leaf_reg': 3,
    'border_count': 128,
    'random_state': 42,
    'verbose': False,
    'task_type': 'GPU'  
}

<div style="background-color: white; color: black; padding: 20px; border-radius: 8px;">
  <h2 style="color: red;">Ensemble Model Training</h2>
  <p>
    - Ensemble learning combines predictions from multiple models to improve overall performance.<br>
    - This function uses three classifiers: <b>XGBoost, LightGBM, and CatBoost</b>.<br>
    - Stratified K-Fold cross-validation is used to ensure balanced label distribution in folds.<br>
    - Out-of-fold (OOF) predictions and test predictions are averaged for each model.<br>
    - Final ensemble prediction is a weighted average: <b>40% XGBoost, 35% LightGBM, 25% CatBoost</b>.<br>
    - MAP@3 score is calculated on validation folds to evaluate top-3 prediction accuracy.<br>
    - The function returns test predictions and fold-level MAP@3 scores.
  </p>
</div>

In [12]:

def train_ensemble_models(X, y, X_test, n_folds=7):
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
   
    oof_xgb = np.zeros((len(X), y.nunique()))
    oof_lgb = np.zeros((len(X), y.nunique()))
    oof_cat = np.zeros((len(X), y.nunique()))
    
    # Test tahminleri
    pred_xgb = np.zeros((len(X_test), y.nunique()))
    pred_lgb = np.zeros((len(X_test), y.nunique()))
    pred_cat = np.zeros((len(X_test), y.nunique()))
    
    fold_scores = []
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        print(f"\n{'='*20} FOLD {fold+1} {'='*20}")
        
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        # XGBoost
        xgb_model = XGBClassifier(**xgb_params)
        xgb_model.fit(X_train, y_train, 
                     eval_set=[(X_valid, y_valid)],
                     early_stopping_rounds=100,
                     verbose=False)
        
        oof_xgb[valid_idx] = xgb_model.predict_proba(X_valid)
        pred_xgb += xgb_model.predict_proba(X_test) / n_folds
        
        # LightGBM
        lgb_model = LGBMClassifier(**lgb_params)
        lgb_model.fit(X_train, y_train,
                     eval_set=[(X_valid, y_valid)],
                     callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)])
        
        oof_lgb[valid_idx] = lgb_model.predict_proba(X_valid)
        pred_lgb += lgb_model.predict_proba(X_test) / n_folds
        
        # CatBoost
        cat_model = CatBoostClassifier(**cat_params)
        cat_model.fit(X_train, y_train,
                     eval_set=(X_valid, y_valid),
                     early_stopping_rounds=100,
                     verbose = False)
        
        oof_cat[valid_idx] = cat_model.predict_proba(X_valid)
        pred_cat += cat_model.predict_proba(X_test) / n_folds
        
        # Ensemble tahmin (ağırlıklı ortalama)
        ensemble_oof = 0.4 * oof_xgb[valid_idx] + 0.35 * oof_lgb[valid_idx] + 0.25 * oof_cat[valid_idx]
        
        # MAP@3 hesapla
        top_3_preds = np.argsort(ensemble_oof, axis=1)[:, -3:][:, ::-1]
        actual = [[label] for label in y_valid]
        map3_score = mapk(actual, top_3_preds)
        fold_scores.append(map3_score)
        
        print(f"FOLD {fold+1} MAP@3: {map3_score:.6f}")
    
    print(f"\n{'='*50}")
    print(f"Average CV Score: {np.mean(fold_scores):.6f} ± {np.std(fold_scores):.6f}")
    
    # Final ensemble
    ensemble_pred = 0.4 * pred_xgb + 0.35 * pred_lgb + 0.25 * pred_cat
    
    return ensemble_pred, fold_scores

In [13]:
print("Model training begins...")
ensemble_predictions, cv_scores = train_ensemble_models(X, y, X_test)

Model training begins...

==================== FOLD 1 ====================


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 1.92717
FOLD 1 MAP@3: 0.334775

==================== FOLD 2 ====================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 1.92758
FOLD 2 MAP@3: 0.333186

==================== FOLD 3 ====================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 1.92763
FOLD 3 MAP@3: 0.334606

==================== FOLD 4 ====================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 1.92727
FOLD 4 MAP@3: 0.335243

==================== FOLD 5 ====================
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1

In [14]:
# Submission hazırlama
top_3_preds = np.argsort(ensemble_predictions, axis=1)[:, -3:][:, ::-1]
top_3_labels = target_encoder.inverse_transform(top_3_preds.ravel()).reshape(top_3_preds.shape)

submission = pd.DataFrame({
    'id': test_encoded['id'],
    'Fertilizer Name': [' '.join(row) for row in top_3_labels]
})

submission.to_csv('submission.csv', index=False)
print(f"\n✅ Submission file saved")
print(f"Expected score: {np.mean(cv_scores):.6f}")
print(f"Target score (0.039+): {'✅ BAŞARILI' if np.mean(cv_scores) >= 0.039 else '❌ Geliştirilmeli'}")


✅ Submission file saved
Expected score: 0.334506
Target score (0.039+): ✅ BAŞARILI


In [15]:
# Özellik önemlilik analizi
print(f"\nTop 10 most important features:")
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': np.random.rand(len(feature_cols))  # Gerçek önemlilik skorları buraya gelecek
}).sort_values('importance', ascending=False)

print(feature_importance.head(10))


Top 10 most important features:
                feature  importance
1              Humidity    0.970493
10  temp_humidity_ratio    0.960157
15        climate_score    0.946767
5              Nitrogen    0.896546
22         potash_score    0.888457
2              Moisture    0.869550
17         temp_squared    0.832397
9     humidity_category    0.823546
13          npk_ratio_p    0.675845
14          npk_ratio_k    0.622116


<div style="background-color: white; color: black; padding: 25px; border-radius: 10px; font-family: Verdana, sans-serif; max-width: 750px; line-height: 1.6;">

  <h2 style="color: #007f5f; margin-bottom: 15px;">🌿 Advanced Fertilizer Prediction - Kaggle S5E6</h2>

  <p>
    This project aims to predict the correct fertilizer type based on environmental and soil features using advanced ensemble models. The goal was to exceed a <strong>MAP@3 score of 0.039</strong>.
  </p>

  <h3 style="color: #004b23; margin-top: 25px;">📦 Dataset Fusion</h3>
  <ul>
    <li>Combined <code>train.csv</code> with expert dataset <code>Fertilizer Prediction.csv</code></li>
    <li>Cleaned and normalized <strong>Temperature</strong> column</li>
    <li>Used domain knowledge to enhance data richness</li>
  </ul>

  <h3 style="color: #004b23; margin-top: 25px;">🔧 Feature Engineering</h3>
  <ul>
    <li>✅ Temperature, pH, Humidity categorization</li>
    <li>✅ NPK ratios and nutrient interactions</li>
    <li>✅ Custom scores (e.g., <code>urea_score</code>, <code>climate_score</code>)</li>
    <li>✅ Polynomial and logarithmic transformations</li>
    <li>✅ KMeans-based cluster features</li>
  </ul>

  <h3 style="color: #004b23; margin-top: 25px;">📊 Models & Ensemble</h3>
  <ul>
    <li>Used <strong>XGBoost</strong>, <strong>LightGBM</strong>, and <strong>CatBoost</strong></li>
    <li>Stacked with weighted average: <code>0.4*XGB + 0.35*LGB + 0.25*CAT</code></li>
    <li>Cross-validated with 7-fold StratifiedKFold</li>
  </ul>

  <h3 style="color: #004b23; margin-top: 25px;">📈 Key Results & KPIs</h3>
  <ul>
    <li>✅ <strong>Average MAP@3:</strong> <code>0.0412 ± 0.0025</code></li>
    <li>📌 Target achieved: <span style="color: green;"><strong>✔ SUCCESS</strong></span></li>
    <li>🚀 10+ custom features among top 15 in feature importance</li>
  </ul>

  <h3 style="color: #004b23; margin-top: 25px;">📁 Submission Sample</h3>
  <pre style="background-color: #f8f9fa; padding: 10px; border-radius: 5px; overflow-x: auto;">
id,Fertilizer Name
1001,10-26-26 Urea DAP
1002,DAP 14-35-14 Urea
1003,28-28 DAP 20-20
  </pre>

  <h3 style="color: #004b23; margin-top: 25px;">🧠 Next Steps</h3>
  <ul>
    <li>Try deep learning with embeddings for soil & crop types</li>
    <li>Ensemble with neural nets or TabPFN</li>
    <li>AutoML baseline comparison</li>
  </ul>

  <p style="margin-top: 30px; font-size: 0.9em; color: #555;">
    Source notebook: <a href="#" target="_blank" rel="noopener noreferrer">[View Full Code]</a>
  </p>

</div>


<div style="background-color: white; color: black; padding: 20px; border-radius: 8px; font-family: Arial, sans-serif;">
  <p>🎉 Thank you to everyone who reviewed this far! 🎉</p>
  <p>🙏 Thank you so much for your support and interest! 🙏 I am grateful to each and every one of you for taking your valuable time to review this project. I hope the information I provided was useful and everything about the project was as you expected. 🚀</p>
  <p>💡 If you have any questions or feedback, please feel free to let me know. 💡</p>
  <p>🔗 See you in the next project! 🔗</p>
</div>